Build P3HT bulk simulation 

In [149]:
import os,sys, copy, pickle

In [2]:
from streamm import *

In [3]:
%load_ext autoreload
%autoreload 2

Load resources from previous example 

In [4]:
res_tag = 'local'  # Change this to remote to run the calculations remotely 
res_local = resource.Resource(res_tag)
res_local.load_json()

Reading in resource json file res_local.json 


In [5]:
peregrine = resource.Resource('peregrine')
peregrine.load_json()

Reading in resource json file res_peregrine.json 


Load oligomer from previous examples 

In [ ]:
p3ht_n10_esp = buildingblock.Container('p3ht_n10_esp')

In [ ]:
p3ht_n10_esp.read_cply()

In [ ]:
p3ht_n10_esp.lat_cubic(50.0)

In [ ]:
p3ht_n10_esp.guess_oplsa()

In [ ]:
p3ht_n10_esp.bonded_bonds()
p3ht_n10_esp.bonded_angles()
p3ht_n10_esp.bonded_dih()

Place the oligomer into a simulation and add parameters 

In [ ]:
lmp_oligo = lammps.LAMMPS('p3ht_n10_esp_nvt')
lmp_oligo.load_json()

In [ ]:
lmp_oligo.set_resource(peregrine)

In [ ]:
lmp_oligo.properties['scratch'] = lmp_oligo.dir['scratch']

In [ ]:
lmp_oligo.make_dir()

In [ ]:
lmp_oligo.add_strucC(p3ht_n10_esp)

In [ ]:
print lmp_oligo.strucC.lat._matrix

In [ ]:
lmp_oligo.read_param("conj.param")

In [ ]:
lmp_oligo.set_ffparam()

In [ ]:
for p,pt in lmp_oligo.strucC.particles.iteritems(): # properties["charge"] 
    print p,pt.properties["symbol"],pt.properties["fftype"]

Read in templates

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_nvt.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_oligo.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_oligo.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
print lmp_oligo.dir['launch']

In [ ]:
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.load_str('templates','run')

In [ ]:
lmp_oligo.load_str('templates','in')

In [ ]:
lmp_oligo.write_data()

In [ ]:
lmp_oligo.replacewrite_prop('in','input','in','%s.in'%(lmp_oligo.tag))

In [ ]:
lmp_oligo.properties['input_in'] = lmp_oligo.files['input']['in']

In [ ]:
lmp_oligo.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_oligo.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_oligo.tag)
lmp_oligo.add_file(file_type,file_key,file_name)

In [ ]:
os.chdir(lmp_oligo.dir['home'])
lmp_oligo.dump_json()
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.push()

In [ ]:
lmp_oligo.run()

In [ ]:
lmp_oligo.check()
print lmp_oligo.meta['status']

In [ ]:
lmp_oligo.analysis()
print lmp_oligo.meta['status']

In [ ]:
print lmp_oligo.files['output']

In [ ]:
os.chdir(lmp_oligo.dir['home'])
lmp_oligo.dump_json()
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.meta['status'] = 'finished'

In [ ]:
lmp_oligo.store()

In [ ]:
print lmp_oligo.files['output']

In [ ]:
lmp_oligo.pull()

In [ ]:
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.read_data(lmp_oligo.files['output']['data_3'])

In [ ]:
lmp_oligo.strucC.write_cply()

In [ ]:
lmp_oligo.strucC.group_prop('mol','mol')

In [ ]:
lmp_oligo.strucC.groupsets['mol'].group_pbcs()

In [ ]:
print lmp_oligo.strucC.tag 

In [ ]:
lmp_oligo.strucC.tag = "%s_nvt"%(lmp_oligo.strucC.tag )

In [ ]:
os.chdir(lmp_oligo.dir['home'])

In [ ]:
lmp_oligo.dump_json()

In [ ]:
lmp_oligo.strucC.write_xyz()

In [ ]:
lmp_oligo.strucC.write_cply()

In [ ]:
p3ht_n10_nvt = copy.deepcopy(lmp_oligo.strucC)

Replicate the oligomer into large simulation cell

In [6]:
xO = 20

In [7]:
p3ht_n10_esp_x = buildingblock.Container('p3ht_n10_x%d'%(xO))
p3ht_n10_esp_x.read_cply()

In [ ]:
p3ht_n10_esp_x.lat_cubic(150.0)

In [ ]:
print p3ht_n10_esp_x.lat._matrix

Set seed 

In [ ]:
s = 2984927

In [ ]:
p3ht_n10_esp_x = p3ht_n10_esp_x.add_struc(p3ht_n10_nvt,xO,s)

In [8]:
p3ht_n10_esp_x.bonded_bonds()
p3ht_n10_esp_x.bonded_angles()
p3ht_n10_esp_x.bonded_dih()

In [ ]:
p3ht_n10_esp_x.tag = 'p3ht_n10_x%d'%(xO)

In [ ]:
p3ht_n10_esp_x.write_xyz()

In [ ]:
p3ht_n10_esp_x.write_cply()

In [ ]:
print p3ht_n10_esp_x.n_bonds,p3ht_n10_esp_x.n_angles,p3ht_n10_esp_x.n_dihedrals

Create LAMMPS calculation to equilibrate bulk system

In [9]:
xO = 20

In [10]:
lmp_bulk = lammps.LAMMPS('p3ht_n10_esp_x%d'%(xO))

In [11]:
lmp_bulk.load_json()

Resource tag found peregrine 
Reading in resource json file res_peregrine.json 


In [ ]:
lmp_bulk.set_resource(peregrine)

In [ ]:
lmp_bulk.properties['scratch'] = lmp_bulk.dir['scratch']

In [ ]:
lmp_bulk.make_dir()

In [ ]:
lmp_bulk.strucC = buildingblock.Container()

In [12]:
lmp_bulk.add_strucC(p3ht_n10_esp_x)

In [13]:
print lmp_bulk.strucC.n_particles

5040


In [14]:
lmp_bulk.read_param("conj.param")

In [15]:
lmp_bulk.set_ffparam()

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_nvt.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_bulk.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_bulk.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
print lmp_bulk.dir['launch']

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.load_str('templates','run')

In [ ]:
lmp_bulk.load_str('templates','in')

In [ ]:
lmp_bulk.write_data()

In [ ]:
lmp_bulk.replacewrite_prop('in','input','in','%s.in'%(lmp_bulk.tag))

In [ ]:
lmp_bulk.properties['input_in'] = lmp_bulk.files['input']['in']

In [ ]:
lmp_bulk.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_bulk.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_bulk.tag)
lmp_bulk.add_file(file_type,file_key,file_name)

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.push()

In [ ]:
lmp_bulk.run()

In [ ]:
lmp_bulk.check()
print lmp_bulk.meta['status']

In [ ]:
lmp_bulk.analysis()
print lmp_bulk.meta['status']

In [ ]:
print lmp_bulk.files['output']

In [ ]:
print lmp_bulk.dir

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.meta['status'] = 'finished'

In [ ]:
lmp_bulk.store()

In [ ]:
lmp_bulk.pull()

In [37]:
os.chdir(lmp_bulk.dir['launch'])

In [17]:
lmp_bulk.read_data_pos(lmp_bulk.files['output']['data_3'])

In [ ]:
pos_nvt = copy.deepcopy(lmp_bulk.strucC.positions)

In [ ]:
lmp_bulk.strucC.write_cply()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [18]:
lmp_bulk.strucC.group_prop('mol','mol')

In [19]:
lmp_bulk.strucC.groupsets['mol'].group_pbcs()

In [33]:
print lmp_bulk.tag 

p3ht_n10_esp_x20


In [21]:
lmp_bulk.strucC.tag = "%s_nvt"%(lmp_bulk.strucC.tag )

Write new data file and copy it to scratch to be stored with output, 
since LAMMPS does not print dihedral angles for opls style 

In [39]:
data_file = '%s.data'%(lmp_bulk.tag)

In [40]:
lmp_bulk.write_data(data_file)

In [41]:
lmp_bulk.add_file('output','data_4',data_file)

In [44]:
ssh_id = "%s@%s"%(lmp_bulk.resource.ssh['username'],lmp_bulk.resource.ssh['address'])
from_pathfile = os.path.join(lmp_bulk.dir['launch'],data_file)
bash_command = "scp %s %s:%s"%(from_pathfile,ssh_id,lmp_bulk.dir['scratch'])
print bash_command
os.system(bash_command)

scp /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_esp_x20/p3ht_n10_esp_x20.data tkemper@peregrine.hpc.nrel.gov:/scratch/tkemper/p3ht_n10_esp_x20/


0

In [46]:
lmp_bulk.meta['status'] = 'finished'
print lmp_bulk.meta['status']

finished


In [47]:
lmp_bulk.store()

runnning store function in /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_esp_x20 
Storing input files 
 file_i  p3ht_n10_esp_x20.data
 file_i  p3ht_n10_esp_x20_input.tgz
 file_i  p3ht_n10_esp_x20.in
> compressed_file  p3ht_n10_esp_x20_input.tgz
Storing scripts files 
 file_i  p3ht_n10_esp_x20.sh
 file_i  p3ht_n10_esp_x20_scripts.tgz
> compressed_file  p3ht_n10_esp_x20_scripts.tgz
Storing output files 
 file_i  p3ht_n10_esp_x20.log
 file_i  min2.data
 file_i  nvt1.data
 file_i  min1.data
 file_i  p3ht_n10_esp_x20_output.tgz
 file_i  p3ht_n10_esp_x20.data
 file_i  nvt1_dump.dcd
> compressed_file  p3ht_n10_esp_x20_output.tgz
No files of type data present


In [35]:
os.chdir(lmp_bulk.dir['home'])

In [36]:
lmp_bulk.dump_json()

{'files': {u'templates': {u'run': u'lammps_peregrine.pbs', u'in': u'lammps_nvt.in'}, u'input': {'data_file': 'p3ht_n10_esp_x20.data', u'compressed': u'p3ht_n10_esp_x20_input.tgz', u'in': u'p3ht_n10_esp_x20.in'}, u'output': {u'log': u'p3ht_n10_esp_x20.log', u'data_3': u'min2.data', u'data_2': u'nvt1.data', u'data_1': u'min1.data', u'compressed': u'p3ht_n10_esp_x20_output.tgz', 'data_4': 'p3ht_n10_esp_x20.data', u'dcd_1': u'nvt1_dump.dcd'}, u'data': {}, u'scripts': {u'run': u'p3ht_n10_esp_x20.sh', u'compressed': u'p3ht_n10_esp_x20_scripts.tgz'}}, 'meta': {u'date': u'2016-11-08T08:23:28.336627', u'status': u'stored', u'resource': u'peregrine', u'software': u'lammps'}, 'references': {}, 'units': {u'distance': u'angstroms', u'energy': u'kcal/mol', u'angle': u'radians', u'time': u'ns'}, 'data': {}, 'properties': {u'queue': u'short', u'compress_sufix': u'tgz', u'comp_key': u'compressed', u'uncompress': u'tar -xzf ', 'data_file': 'p3ht_n10_esp_x20.data', u'scratch': u'/scratch/tkemper', u'comp

In [26]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.write_cply()

In [25]:
lmp_bulk.write_data()

In [ ]:
print lmp_bulk.tag

Run Equilibration 

In [109]:
lmp_equ = lammps.LAMMPS('p3ht_n10_x20_equ')

In [110]:
lmp_equ.set_resource(peregrine)

In [111]:
lmp_equ.properties['scratch'] = lmp_equ.dir['scratch']

In [112]:
lmp_equ.make_dir()

In [113]:
lmp_equ.add_refcalc(lmp_bulk)

In [114]:
lmp_equ.properties['nodes'] = 4
lmp_equ.properties['nproc'] = lmp_equ.properties['nodes']*lmp_equ.properties['ppn']
print lmp_equ.properties

{'uncompress': 'tar -xzf ', u'scratch': u'/scratch/tkemper/p3ht_n10_x20_equ/', 'compress': 'tar -czf ', u'ppn': 24, 'finish_str': 'Loop time of', u'allocation': u'orgopv', u'nproc': 96, u'pmem': 1500, 'compress_sufix': 'tgz', u'feature': u'24core', u'queue': u'short', u'e-mail': u'travis.kemper@nrel.gov', 'comp_key': 'compressed', u'nodes': 4, u'walltime': 4, u'exe_command': u'qsub '}


In [115]:
lmp_equ.files['input']['data_file'] = lmp_bulk.files['output']['data_4']
print lmp_equ.files['input']['data_file']

p3ht_n10_esp_x20.data


In [116]:
lmp_equ.properties['data_file']  = lmp_equ.files['input']['data_file'] 

In [117]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_equ1.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_equ.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates in lammps_equ1.in templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/lammps_equ1.in to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ/lammps_equ1.in 


In [118]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_equ.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates run lammps_peregrine.pbs templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/lammps_peregrine.pbs to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ/lammps_peregrine.pbs 


In [119]:
print lmp_equ.dir['launch']

/Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ/


In [120]:
os.chdir(lmp_equ.dir['launch'])

In [121]:
lmp_equ.load_str('templates','run')

In [122]:
lmp_equ.load_str('templates','in')

In [123]:
lmp_equ.replacewrite_prop('in','input','in','%s.in'%(lmp_equ.tag))

In [124]:
lmp_equ.properties['input_in'] = lmp_equ.files['input']['in']

In [125]:
lmp_equ.replacewrite_prop('run','scripts','run','%s.pbs'%(lmp_equ.tag))

In [126]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_equ.tag)
lmp_equ.add_file(file_type,file_key,file_name)

In [127]:
os.chdir(lmp_equ.dir['home'])
lmp_equ.dump_json()
os.chdir(lmp_equ.dir['launch'])

{'files': {'templates': {'run': 'lammps_peregrine.pbs', 'in': 'lammps_equ1.in'}, 'input': {'data_file': 'p3ht_n10_esp_x20.data', 'in': 'p3ht_n10_x20_equ.in'}, 'output': {'log': 'p3ht_n10_x20_equ.log'}, 'data': {}, 'scripts': {'run': 'p3ht_n10_x20_equ.pbs'}}, 'meta': {'date': '2016-11-08T11:43:25.026199', 'status': 'written', 'resource': 'peregrine', 'software': 'lammps'}, 'references': {'p3ht_n10_esp_x20': 'p3ht_n10_esp_x20'}, 'units': {'distance': 'angstroms', 'energy': 'kcal/mol', 'angle': 'radians', 'time': 'ns'}, 'data': {}, 'properties': {'uncompress': 'tar -xzf ', u'scratch': u'/scratch/tkemper/p3ht_n10_x20_equ/', 'compress': 'tar -czf ', u'ppn': 24, 'finish_str': 'Loop time of', u'allocation': u'orgopv', 'run_list': '', u'nproc': 96, 'input_in': 'p3ht_n10_x20_equ.in', u'pmem': 1500, 'compress_sufix': 'tgz', 'data_file': 'p3ht_n10_esp_x20.data', u'feature': u'24core', u'queue': u'short', u'e-mail': u'travis.kemper@nrel.gov', 'comp_key': 'compressed', u'nodes': 4, u'walltime': 4, 

In [128]:
ref_calc = lmp_equ.references['p3ht_n10_esp_x20']

In [129]:
print ref_calc.files

{u'templates': {u'run': u'lammps_peregrine.pbs', u'in': u'lammps_nvt.in'}, u'input': {'data_file': 'p3ht_n10_esp_x20.data', u'compressed': u'p3ht_n10_esp_x20_input.tgz', u'in': u'p3ht_n10_esp_x20.in'}, u'output': {u'log': u'p3ht_n10_esp_x20.log', u'data_3': u'min2.data', u'data_2': u'nvt1.data', u'data_1': u'min1.data', u'compressed': u'p3ht_n10_esp_x20_output.tgz', 'data_4': 'p3ht_n10_esp_x20.data', u'dcd_1': u'nvt1_dump.dcd'}, u'data': {}, u'scripts': {u'run': u'p3ht_n10_esp_x20.sh', u'compressed': u'p3ht_n10_esp_x20_scripts.tgz'}}


In [130]:
lmp_equ.push()

 Resource type ssh 
runnning push function in /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ 
Compressing and copying input files to scratch directory 
 file_i  p3ht_n10_esp_x20.data
 file_i  p3ht_n10_x20_equ.in
> compressed_file  p3ht_n10_x20_equ_input.tgz
Compressing and copying templates files to scratch directory 
 file_i  lammps_peregrine.pbs
 file_i  lammps_equ1.in
> compressed_file  p3ht_n10_x20_equ_templates.tgz
Compressing and copying scripts files to scratch directory 
 file_i  p3ht_n10_x20_equ.pbs
> compressed_file  p3ht_n10_x20_equ_scripts.tgz
Copying output of reference calculations p3ht_n10_esp_x20


In [131]:
lmp_equ.run()

Calculation with status written 
Resource type ssh 
Executing run command ssh tkemper@peregrine.hpc.nrel.gov ' cd /scratch/tkemper/p3ht_n10_x20_equ/ ; qsub p3ht_n10_x20_equ.pbs '  


In [ ]:
lmp_equ.check()
print lmp_equ.meta['status']

In [ ]:
lmp_bulk.analysis()
print lmp_bulk.meta['status']

In [ ]:
print lmp_bulk.files['output']

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.meta['status'] = 'finished'

In [ ]:
lmp_bulk.store()

In [ ]:
lmp_bulk.pull()

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.read_data(lmp_bulk.files['output']['data_3'])

In [ ]:
lmp_bulk.strucC.write_cply()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.group_prop('mol','mol')

In [ ]:
lmp_bulk.strucC.groupsets['mol'].group_pbcs()

In [ ]:
print lmp_bulk.strucC.tag 

In [ ]:
lmp_bulk.strucC.tag = "%s_nvt"%(lmp_bulk.strucC.tag )

In [77]:
os.chdir(lmp_bulk.dir['home'])

In [ ]:
lmp_equ.dump_json()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [85]:
lmp_bulk.strucC.write_cply()

Analyize bulk properties with streamm

In [90]:
import pandas as pd

In [78]:
print lmp_equ.tag

p3ht_n10_x20_equ2


In [79]:
os.getcwd()

'/Users/tkemper/Development/streamm_ext/streamm-tools/examples'

In [80]:
equ_rdf = calculation.CalculationRes('%s_rdf1'%(lmp_equ.tag))

In [81]:
equ_rdf.set_resource(peregrine)

In [82]:
equ_rdf.properties['scratch'] = equ_rdf.dir['scratch']

In [83]:
equ_rdf.add_refcalc(lmp_bulk)

In [84]:
equ_rdf.make_dir()

Making /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ2_rdf1/ 


In [138]:
os.chdir(equ_rdf.dir['launch'])

In [94]:
file_type = 'input'
file_key = 'cply'
file_name = "p3ht_n10_x20_nvt.cply"
from_dirkey = 'home'
to_dirkey = 'launch'
equ_rdf.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  input cply p3ht_n10_x20_nvt.cply home launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/examples/p3ht_n10_x20_nvt.cply to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ2_rdf1/p3ht_n10_x20_nvt.cply 


In [95]:
p3ht_n10_x20_nvt = buildingblock.Container('p3ht_n10_x20_nvt')

In [96]:
p3ht_n10_x20_nvt.read_cply()

In [97]:
p3ht_n10_x20_nvt.group_prop('residue','residue')

Create pandas data frame of particle properties 

In [98]:
p3ht_n10_x20_nvt.propcompile_particles()

In [99]:
part_df =  pd.DataFrame(p3ht_n10_x20_nvt.prop_particles)

In [101]:
syCA = part_df['fftype'] == 'CA'
syS = part_df['fftype'] == 'S'

In [102]:
sub_i = part_df[ syCA | syS ]

In [103]:
struc_i = p3ht_n10_x20_nvt.getSubStructure(sub_i.index,'sel_SCA') 

In [104]:
struc_i.write_xyz()

In [134]:
syCT = part_df['fftype'] == 'CT'

In [135]:
sub_j = part_df[ syCT ]

In [136]:
struc_j = p3ht_n10_x20_nvt.getSubStructure(sub_j.index,'sel_CT') 

In [139]:
struc_j.write_xyz()

Check that selections are what you expect and save lists 

In [160]:
list_i = [str(pkey) for pkey in sub_i.index]

In [157]:
outfile = open('list_i','wb')
outfile.write("\n".join(list_i))
outfile.close()

In [165]:
equ_rdf.add_file('input','list_i','list_i')

In [163]:
list_j = [str(pkey) for pkey in sub_j.index]

In [164]:
outfile = open('list_j','wb')
outfile.write("\n".join(list_j))
outfile.close()

In [166]:
equ_rdf.add_file('input','list_j','list_j')

Find pairs 

In [140]:
pairs_ii = p3ht_n10_x20_nvt.find_pairs(sub_i.index,sub_i.index)

In [145]:
pairs_ii_inter = p3ht_n10_x20_nvt.find_pairs(sub_i.index,sub_i.index,mol_inter=True)

In [147]:
pairs_ii_intra = p3ht_n10_x20_nvt.find_pairs(sub_i.index,sub_i.index,mol_intra=True)

In [148]:
print pairs_ii_intra

[[ 0.  1.  1. ...,  0.  0.  0.]
 [ 1.  0.  1. ...,  0.  0.  0.]
 [ 1.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  1.]
 [ 0.  0.  0. ...,  1.  0.  1.]
 [ 0.  0.  0. ...,  1.  1.  0.]]
